<a href="https://colab.research.google.com/github/EmmaJaneShaw/Favorita/blob/main/favorita_joined_processed_test_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy
!pip install pandas
!pip install seaborn
!pip install scikit-learn
!pip install scipy
!pip install patsy
!pip install statsmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as dt
import os #interact with operating system
from pandas import set_option #view more of the dataframe
import random
import sys


In [47]:
stores = pd.read_csv('/content/drive/MyDrive/Favorita/stores.csv')
items = pd.read_csv('/content/drive/MyDrive/Favorita/items.csv')
oil = pd.read_csv('/content/drive/MyDrive/Favorita/oil.csv', parse_dates=['date'])
holiday_events = pd.read_csv('/content/drive/MyDrive/Favorita/holidays_events.csv', parse_dates=['date'])
transactions = pd.read_csv('/content/drive/MyDrive/Favorita/transactions.csv', parse_dates=['date'])
test = pd.read_csv('/content/drive/MyDrive/Favorita/test.csv', parse_dates=['date'])

In [66]:
test = test.set_index("date")

In [67]:
test["month"] = test.index.month
test["week"] = test.index.week
test["day"] = test.index.weekday
        # reset index
test.reset_index(inplace=True)
# new dataframe
test.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  


,date,id,store_nbr,item_nbr,onpromotion,month,week,day
0,2017-08-16,125497040,1,96995,False,8,33,2
1,2017-08-16,125497041,1,99197,False,8,33,2
2,2017-08-16,125497042,1,103501,False,8,33,2
3,2017-08-16,125497043,1,103520,False,8,33,2
4,2017-08-16,125497044,1,103665,False,8,33,2


In [68]:
#join dataframes
import datetime as dt
from sklearn.base import BaseEstimator, TransformerMixin
class join_frames(BaseEstimator, TransformerMixin):
    def __init__(self):
        print("join_frames -> init")
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        test_stores = X[0].merge(X[1], right_on = 'store_nbr', left_on='store_nbr')
        test_stores_oil = test_stores.merge(X[2], right_on='date', left_on='date')
        test_stores_oil_items = test_stores_oil.merge(X[3], right_on = 'item_nbr', left_on = 'item_nbr')
        test_stores_oil_items_transactions = test_stores_oil_items.merge(X[4], right_on = ['date', 'store_nbr'], left_on = ['date', 'store_nbr'])
        test_stores_oil_items_transactions_holiday = test_stores_oil_items_transactions.merge(X[5], right_on = 'date', left_on = 'date')
        
        joined_test = test_stores_oil_items_transactions_holiday.copy(deep = True)
        return joined_test

In [69]:
from sklearn.pipeline import Pipeline
pipe_processing = Pipeline([
        ('join_frames', join_frames())
    ])

# our prepared data
joined_test = pipe_processing.fit_transform([test, stores, oil, items, transactions, holiday_events])
joined_test.head()

join_frames -> init


,id,store_nbr,item_nbr,onpromotion,month,week,day,city,state,type_x,...,family,class,perishable,transactions,date,type_y,locale,locale_name,description,transferred


In [70]:
import datetime as dt
from sklearn.base import BaseEstimator, TransformerMixin
class prepare_data(BaseEstimator, TransformerMixin):
    def __init__(self):
        print("prepare_data -> init")
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        test_df = test.copy(deep = True)
               
        # change the bool to int
        test_df['onpromotion'] = test_df['onpromotion'].astype(int)
        test_df['transferred'] = test_df['transferred'].astype(int)

        # change the names
        test_df.rename(columns={'type_x': 'st_type', 'type_y': 'hol_type'}, inplace=True)

        # drop the id
        test_df.drop(['id'], axis=1, inplace=True)
        
        print(dtest_df.head())
        
        # handle date
        test_df['date'] = pd.to_datetime(test_df['date'])
        test_df['date'] = test_df['date'].map(dt.datetime.toordinal)
                
        return test_df

In [71]:
# split dataframe into numerical values, categorical values and date
class split_data(BaseEstimator, TransformerMixin):
    def __init__(self):
        print("split_data -> init")
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        # Get columns for each type         
        df_ = X.drop(['date'], axis = 1)
        cols = df_.columns
        num_cols = df_._get_numeric_data().columns
        cat_cols = list(set(cols) - set(num_cols))
        
        data_num_df = X[num_cols]
        data_cat_df = X[cat_cols]
        data_date_df = X['date']
        
        return data_num_df, data_cat_df, data_date_df

In [72]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

class process_data(BaseEstimator, TransformerMixin):
    def __init__(self):
        print("process_data -> init")
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        ### numerical data
        # impute nulls in numerical attributes
        imputer = SimpleImputer(strategy="mean", copy="true")
        num_imp = imputer.fit_transform(X[0])
        data_num_df = pd.DataFrame(num_imp, columns=X[0].columns, index=X[0].index)
        
        # apply standard scaling
        scaler = StandardScaler()
        scaler.fit(data_num_df)
        num_scaled = scaler.transform(data_num_df)
        data_num_df = pd.DataFrame(num_scaled, columns=X[0].columns, index=X[0].index)
        
        ### categorical data
        # one hot encoder
        cat_encoder = OneHotEncoder(sparse=False)
        data_cat_1hot = cat_encoder.fit_transform(X[1])
        
        # convert it to datafram with n*99 where n number of rows and 99 is no. of categories
        data_cat_df = pd.DataFrame(data_cat_1hot, columns=cat_encoder.get_feature_names()) #, index=X[1].index)
                
        return data_num_df, data_cat_df, X[2]

In [64]:
class join_df(BaseEstimator, TransformerMixin):
    def __init__(self):
        print("join_df -> init")
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        ### numerical data
        test_df = X[0].join(X[1])
        test_df = test_df.join(X[2])
        
        return test_df

In [65]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler


pipe_processing2 = Pipeline([
        ('join_frames', join_frames()),
        ('prepare_data', prepare_data()),                     
        ('split_data', split_data()),
        ('process_data', process_data()),
        ('join_data', join_df())
    ])


test_df = pipe_processing2.fit_transform(test)

join_frames -> init
prepare_data -> init
split_data -> init
process_data -> init
join_df -> init


KeyError: ignored